In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
baseURL = 'https://www.bbc.com'

In [5]:
request = requests.get('https://www.bbc.com/urdu')

In [6]:
request.status_code

200

In [7]:
data = None

In [8]:
if request.status_code == 200:
    data = request.text

In [9]:
soup = BeautifulSoup(data, 'html5lib')

In [10]:
soup.prettify()

'<!DOCTYPE html>\n<html class="no-js" dir="rtl" lang="ur">\n <head>\n  <meta content="IE=edge" data-react-helmet="true" http-equiv="X-UA-Compatible"/>\n  <meta charset="utf-8" data-react-helmet="true"/>\n  <meta content="noodp,noydir" data-react-helmet="true" name="robots"/>\n  <meta content="#B80000" data-react-helmet="true" name="theme-color"/>\n  <meta content="width=device-width, initial-scale=1, minimum-scale=1" data-react-helmet="true" name="viewport"/>\n  <meta content="BBC News اردو" data-react-helmet="true" name="apple-mobile-web-app-title"/>\n  <meta content="BBC News اردو" data-react-helmet="true" name="application-name"/>\n  <meta content="تازہ ترین خبروں، ویڈیوز اور آڈیوز کے لیے بی بی سی اردو پر آئیے۔ بی بی سی اردو دنیا بھر کی خبروں کے حصول کے لیے ایک قابلِ اعتماد ویب سائٹ ہے۔" data-react-helmet="true" name="description"/>\n  <meta content="100004154058350" data-react-helmet="true" property="fb:admins"/>\n  <meta content="1609039196070050" data-react-helmet="true" property

In [11]:
sections_soup = soup.find_all('a', {'class' : 'StyledLink-sc-158x7z5-2'})
sections = []
for section in sections_soup:
    if section.text ==  'ویڈیو' or section.text == 'پنجابی ويڈیو':
        continue
    sections.append(section)


In [12]:
sections

[<a class="StyledLink-sc-158x7z5-2 dxndRL" href="/urdu"><span class="StyledSpan-sc-158x7z5-4 ktTFbu" role="text"><span class="VisuallyHiddenText-sc-1yjwwa5-0 ldFJPH">موجودہ صفحہ<!-- -->, </span>صفحۂ اول</span></a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/pakistan">پاکستان</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/regional">آس پاس</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/world">ورلڈ</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/sport">کھیل</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/entertainment">فن فنکار</a>,
 <a class="StyledLink-sc-158x7z5-2 eVHHC" href="/urdu/science">سائنس</a>]

In [13]:
for section in sections:
    url = baseURL+section.get('href')
    sec_stories = BeautifulSoup(requests.get(url).text, 'html.parser')
    print(url)

https://www.bbc.com/urdu
https://www.bbc.com/urdu/pakistan
https://www.bbc.com/urdu/regional
https://www.bbc.com/urdu/world
https://www.bbc.com/urdu/sport
https://www.bbc.com/urdu/entertainment
https://www.bbc.com/urdu/science


In [41]:
dataset = {}
story_no = 0

In [42]:
for section in sections:
    url = baseURL+section.get('href')
    count = 0
    while(True):
        sec_stories = BeautifulSoup(requests.get(url).text, 'html.parser')
        if(section.text == 'آس پاس'):
            stories = sec_stories.find_all('div', {'class':'eagle-item faux-block-link'})
        else:
            stories = sec_stories.find_all('li', {'class':['StoryPromoLiBase-sc-171lqjd-0','lx-stream__post-container']})
        heading = "Heading: "
        summary = "Summery: "
        for story in stories: 
            if(story != None):
                count += 1
                if section.text == 'موجودہ صفحہ, صفحۂ اول':
                    heading = story.find('a',{'class':'StyledLink-sc-16i2p1z-2'}).text
                elif section.text == 'آس پاس':
                    heading = story.find('h3',{'class':'title-link__title'}).span.text
                else:
                    heading = story.find('a',{'class':'qa-heading-link'}).span.text
                if section.text == 'موجودہ صفحہ, صفحۂ اول':
                    summary = story.find('p',{'class':'Summary-sc-1dvfmi3-4'}).text
                elif section.text == 'آس پاس':
                    summary = story.find('p',{'class':'eagle-item__summary'}).text
                else:
                    summary = story.find('p', {'class':['lx-media-asset-summary', 'qa-story-summary']}).text
            story_no += 1
            dataset[story_no] = [section.text, heading, summary]
            print("Section: ", section.text)
            print("Heading:",heading)
            print("Summary: ",summary)
            print(count, " Stories")
            if count >=25 and section.text ==  'موجودہ صفحہ, صفحۂ اول':
                break
        next = sec_stories.find('a',{'rel':'next'})
        if next and next.get('href') and count<30:
            url = baseURL+next.get('href')
        else:
            break

Section:  موجودہ صفحہ, صفحۂ اول
Heading: کابل یونیورسٹی پر حملے میں کم از کم 19 افراد ہلاک
Summary:  افغان وزارت داخلہ کے ترجمان کے مطابق افغانستان کے دارالحکومت کابل میں کابل یونیورسٹی میں ایرانی کتب کے میلے کے افتتاح سے قبل ہونے والے حملے میں کم از کم 19 افراد ہلاک جبکہ 22 زخمی ہوئے ہیں۔
1  Stories
Section:  موجودہ صفحہ, صفحۂ اول
Heading: بریگیڈیئر اعجاز شاہ اور آسیب کا سایہ
Summary:  کراچی میں امریکی صحافی ڈینیئل پرل کے اغوا کے بعد جب پورے ملک کی پولیس اور خفیہ ادارے عمر شیخ کو تلاش کر رہے تھے تو اعجاز شاہ نے اسے اپنے ہی گھر سے برآمد کر لیا تھا: محمد حنیف کا کالم
2  Stories
Section:  موجودہ صفحہ, صفحۂ اول
Heading: کراچی سے لاپتہ ہونے والی ’مسیحی لڑکی کو بازیاب کرا لیا گیا‘
Summary:  ترجمان سندھ حکومت مرتضیٰ وہاب نے ایک ٹویٹ میں بتایا ہے کہ کراچی سے لاپتہ ہونے والی مسیحی لڑکی کو بازیاب کرا لیا گیا ہے۔
3  Stories
Section:  موجودہ صفحہ, صفحۂ اول
Heading: امریکہ سے نکالا جانے والا سائنسدان جس نے چین کو خلا میں پہنچا دیا
Summary:  یہ قصہ ہے اس چینی سائنسدان کا جس نے نہ صرف ایک بلکہ دو سپ

Section:  پاکستان
Heading: معدومیت کی شکار بلائنڈ انڈس ڈولفن کو شکار کرنے والا ماہی گیر گرفتار
Summary:  سندھ وائلڈ لائف کے ڈائریکٹر کا کہنا تھا کہ ستر اور ساٹھ کی دہائی میں انڈس ڈولفن کی آماجگاہیں اٹک پر دریائے سندھ کے مقام سے شروع ہو جاتی تھیں۔ تاریخی طور پر انڈس ڈولفن دریائے ستلج، جہلم، راوی اور چناب میں بھی پائی جاتی تھیں۔
11  Stories
Section:  پاکستان
Heading: مسلم لیگ ن کے رہنما عبدالقادر بلوچ کی ناراضی کی وجہ کیا ہے؟
Summary:  حزب اختلاف کی جماعتوں کا سیاسی اتحاد پاکستان ڈیموکریٹک موومنٹ میں شامل جنرل (ریٹائرڈ) عبد القادر بلوچ کیا مسلم لیگ ن کے قائد میاں نواز شریف کی جانب سے فوج کی اعلیٰ قیادت کو تنقید کا نشانہ بنانے پر ناراض ہیں یا اس کی وجہ کچھ اور ہی ہے؟
12  Stories
Section:  پاکستان
Heading: صدر ٹرمپ نے پاکستان کا ذکر کب کب اور کہاں کہاں کیا؟
Summary:  امریکی صدر ڈونلڈ ٹرمپ نے اپنے دور صدارت میں نو ایسی ٹویٹس پوسٹ کی ہیں جن میں پاکستان کا ذکر ہے، اور ان میں سے ایک ایسی ہے جو پوسٹ کرنے کے بعد ڈیلیٹ کردی گئی تھی۔ اس کے علاوہ انھوں نے پچاس بار مختلف موقعوں پر پاکستان کا ذکر کیا

Section:  ورلڈ
Heading: امریکہ سے نکالا جانے والا سائنسدان جس نے چین کو خلا میں پہنچا دیا
Summary:  یہ قصہ ہے اس چینی سائنسدان کا جس نے نہ صرف ایک بلکہ دو سپر پاورز کو چاند تک پہنچانے میں مدد کی لیکن آج انھیں صرف ایک ہی ملک میں یاد کیا جاتا ہے۔
1  Stories
Section:  ورلڈ
Heading: آرمینیا نے روس سے مدد مانگی تو ترک وزیر خارجہ آذربائیجان پہنچ گئے
Summary:  ترکی نے ایک بار پھر اس بات پر زور دیا ہے کہ آرمینیا اور آذربائیجان کے مابین جاری محاذ آرائی میں ترکی اپنے اتحادی آذربائیجان کے ساتھ مضبوطی سے کھڑا ہے۔
2  Stories
Section:  ورلڈ
Heading: جنیوا: کم سے کم اجرت کی بلند ترین سطح متعارف کروانے والا دنیا کا پہلا شہر
Summary:  اب جنیوا میں کم سے کم تنخواہ 23 سوئس فرانک فی گھنٹہ ہو گی جو کہ 19 برطانوی پاؤنڈ، 25 امریکی ڈالر یا 22 یورو کے برابر ہے۔ اس حساب سے کسی بھی ملازم کی اب کم سے کم ماہانہ تنخواہ چار ہزار فرانک بنتی ہے۔
3  Stories
Section:  ورلڈ
Heading: مسلم ممالک کی خواتین نے فرانس کی مصنوعات کے بائیکاٹ کا فیصلہ کیوں کیا؟
Summary:  دنیا بھر میں مسلم ممالک میں فرانس کی جانب سے پیغمبر اسلام ک

Section:  کھیل
Heading: زمبابوے کی قسمت کب کھلے گی؟
Summary:  اگر زمبابوے کی ٹیم 28 سال تک انٹرنیشنل کرکٹ کونسل کی رکن رہ کر بھی ایک قدم آگے نہیں بڑھ سکی تو اس سے یقیناً کونسل کی اپنی قابلیت بھی مشکوک ٹھہرتی ہے: پڑھیے پاکستان اور زمبابوے کی سیریز پر سمیع چوہدری کا کالم۔
1  Stories
Section:  کھیل
Heading: پاکستانی امپائر علیم ڈار کا ایک اور عالمی ریکارڈ
Summary:  پاکستانی کرکٹ امپائر علیم ڈار آج پاکستان اور زمبابوے کے درمیان راولپنڈی میں دوسرے ون ڈے انٹرنیشنل میں امپائرنگ کے لیے میدان میں اترے تو یہ ان کا 210 واں ون ڈے انٹرنیشنل میچ تھا جو ایک ریکارڈ ہے۔
2  Stories
Section:  کھیل
Heading: ٹیلر کی کہانی کا ’اینٹی کلائمیکس‘
Summary:  برینڈن ٹیلر اپنے تجربے کی روشنی میں ٹیم کی کشتی گہرے پانیوں سے نکال کر دھیرے دھیرے ساحل کے قریب لے آئے مگر اچانک دوسرے کنارے پر ان کے ساتھی یکے بعد دیگرے ڈگمگانے لگے اور پھر وہ اس دباؤ میں ایسے کھوئے کہ شاہین آفریدی کو میچ لے اڑنے کا موقع مل گیا۔
3  Stories
Section:  کھیل
Heading: پاکستانی بلے بازوں کے رن آؤٹ پر مایوسی بھی، مزاح بھی
Summary:  پچیسویں اوور کی 

Section:  فن فنکار
Heading: جیمز بانڈ کا کردار نبھانے والے شان کونری 90 برس کی عمر میں وفات پا گئے
Summary:  جیمز بانڈ کا کردار نبھانے والے سر شان کونری 90 سال کی عمر میں وفات پا گئے ہیں۔ ان کی وفات کی تصدیق ان کے گھر والوں نے کی ہے۔
1  Stories
Section:  فن فنکار
Heading: سری لنکن کرکٹر مرلی دھرن کی زندگی پر بننے والی فلم تنازعات کا شکار کیوں؟
Summary:  سری لنکا کے سابق آف سپنر متھیا مرلی دھرن کا کہنا ہے کہ ان کی زندگی پر مبنی فلم تنازعات اور تنقید کے باوجود ضرور بنائی جائے گی۔ بی بی سی کی نالینی سواتھاسن نے اس حوالے سے مرلی دھرن سے گفتگو کی۔
2  Stories
Section:  فن فنکار
Heading: پاکستانی اداکار علی خان اب برطانوی فلم میں نظر آئیں گے
Summary:  پاکستانی ڈراموں اور فلموں سے شہرت حاصل کرنے والے اداکار علی خان نئی برطانوی فلم 'مغل موگلی' میں اہم کردار نبھا رہے ہیں۔ یہ فلم 30 اکتوبر کو برطانیہ اور آئرلینڈ کے سنیما گھروں میں نمائش کے لیے پیش کی جائے گی۔
3  Stories
Section:  فن فنکار
Heading: کم کارڈیشیئن کی نجی جزیرے پر ’عاجزانہ‘ سالگرہ پارٹی پر کڑی تنقید
Summary:  ریئلٹی ٹی وی سٹار کا کہنا

Section:  سائنس
Heading: امریکہ سے نکالا جانے والا سائنسدان جس نے چین کو خلا میں پہنچا دیا
Summary:  یہ قصہ ہے اس چینی سائنسدان کا جس نے نہ صرف ایک بلکہ دو سپر پاورز کو چاند تک پہنچانے میں مدد کی لیکن آج انھیں صرف ایک ہی ملک میں یاد کیا جاتا ہے۔
1  Stories
Section:  سائنس
Heading: چین کے 'کاربن سِنک' علاقے ماحول کے لیے کیسے مددگار ثابت ہو رہے ہیں
Summary:  چینی حکومت کی ملک میں تیزی سے درخت لگانے کی پالیسی ملک میں ماحولیات کی بہتری کے لیے نہایت کارگر ثابت ہو رہی ہے۔
2  Stories
Section:  سائنس
Heading: آپ کی جنسی زندگی کو بہتر بنانے کا وعدہ کرنے والی چند ایپس
Summary:  جب 27 سالہ سچن راؤل کی اپنے ساتھی سے تین سال قبل علیحدگی ہوئی تو انھیں ایک ایسے جنسی مسئلے کا سامنا کرنا پڑا جس سے انھیں خاصی ’پریشانی‘ ہوئی۔
3  Stories
Section:  سائنس
Heading: پاکستان سمیت مختلف ممالک کورونا وائرس کی ’دوسری لہر‘ کا مقابلہ کیسے کر رہے ہیں؟
Summary:  جانز ہاپکنز یونیورسٹی کے اعداد و شمار کے مطابق اب تک دنیا بھر میں 46,312,467 افراد میں کورونا کی تشخیص ہوئی ہے جبکہ اموات کی تعداد 1,198,000 ہے۔ بعض یورپی مم

In [28]:
dataset

{'موجودہ صفحہ, صفحۂ اول': 'Heading: امریکی صدارتی انتخاب میں کتنا خرچہ آتا ہے اور یہ کون اٹھاتا ہے؟ Summary: امریکی صدارتی انتخاب کی مہمات اصل انتخاب سے کئی سال پہلے شروع ہوجاتی ہیں۔ کورونا وائرس کی وجہ سے رواں سال یہ انتخابی مہمات مختلف طریقے سے چلائی گئیں، تاہم 3 نومبر کو ہونے والے انتخاب سے قبل بہت بھاری رقوم خرچ کی گئیں۔ امریکی صدارتی انتخاب میں اندازاً کتنا خرچہ آتا ہے اور یہ خرچہ اٹھاتا کون ہے، جاننے کے لیے دیکھیے یہ ڈیجیٹل ویڈیو۔',
 'پاکستان': 'Heading: کیا پاکستان سعودی عرب کو چھوڑ کر ترکی سے قربتیں بڑھا رہا ہے؟ Summary: سفارتی حلقوں میں کئی مہینوں سے یہ بازگشت جاری ہے کہ پاکستان کے خارجہ تعلقات میں ایک ’پالیسی شفٹ‘ آئی ہے اور اب اسلام آباد سعودی عرب کے مقابلے میں ترکی کو فوقیت دے رہا ہے۔',
 'آس پاس': 'Heading: ایک لڑکی کی انٹرنیٹ پر اپنی ’قابل اعتراض‘ تصاویر ہٹوانے کی کوشش Summary: جب انڈیا میں دلی کی ایک لڑکی نے اپنی عریاں تصاویر انٹرنیٹ اور سوشل میڈیا سے ہٹوانے کے لیے عدالت سے رجوع کیا تو ان کے حق میں فیصلہ جاری کیا گیا۔ ہم ان کی کہانی سے کیا سیکھ سکتے ہیں؟',
 'ورلڈ': 'Headi

In [55]:
df = pd.DataFrame.from_dict(dataset, orient='index', columns=['Label', 'Heading', 'Summary'])

In [60]:
df

Label                                            Heading  \
1    موجودہ صفحہ, صفحۂ اول  کابل یونیورسٹی پر حملے میں کم از کم 19 افراد ہلاک   
2    موجودہ صفحہ, صفحۂ اول               بریگیڈیئر اعجاز شاہ اور آسیب کا سایہ   
3    موجودہ صفحہ, صفحۂ اول  کراچی سے لاپتہ ہونے والی ’مسیحی لڑکی کو بازیاب...   
4    موجودہ صفحہ, صفحۂ اول  امریکہ سے نکالا جانے والا سائنسدان جس نے چین ک...   
5    موجودہ صفحہ, صفحۂ اول  پنجاب: چوہان سے اطلاعات کا قلمدان لے لیا گیا، ...   
6    موجودہ صفحہ, صفحۂ اول  آپ کی جنسی زندگی کو بہتر بنانے کا وعدہ کرنے وا...   
7    موجودہ صفحہ, صفحۂ اول  آرمینیا نے روس سے مدد مانگی تو ترک وزیر خارجہ ...   
8    موجودہ صفحہ, صفحۂ اول  جب پاکستان کے پہلے صدر کو ’ملک کے مفاد میں‘ جل...   
9    موجودہ صفحہ, صفحۂ اول  چین کے 'کاربن سِنک' علاقے ماحول کے لیے کیسے مد...   
10   موجودہ صفحہ, صفحۂ اول            امریکہ میں عوام اپنا صدر کیسے چنتے ہیں؟   
11   موجودہ صفحہ, صفحۂ اول  امریکی الیکشن 2020: ڈونلڈ ٹرمپ آگے ہیں یا جو ب...   
12   موجودہ صفحہ, صفحۂ اول  امریکی الیکشن کے نتائج آنے میں تاخیر کیوں ہو س...   
13   موجودہ صفحہ, صفحۂ اول        امریکہ کا اگلا صدر کون ہو گا؟ آپ فیصلہ کریں   
14   موجودہ صفحہ, صفحۂ اول    امریکہ میں ووٹ ڈالنا اتنا مشکل کیوں ہو جاتا ہے؟   
15   موجودہ صفحہ, صفحۂ اول  صدر ٹرمپ نے پاکستان کا ذکر کب کب اور کہاں کہاں...   
16   موجودہ صفحہ, صفحۂ اول                    کیا امریکہ واقعی آزاد ریاست ہے؟   
17   موجودہ صفحہ, صفحۂ اول  امریکی صدر ڈونلڈ ٹرمپ نے دنیا کو کس طرح تبدیل ...   
18   موجودہ صفحہ, صفحۂ اول  جو بائیڈن یا ٹرمپ: کس کی جیت پاکستان کے حق میں...   
19   موجودہ صفحہ, صفحۂ اول  مسلم ممالک کی خواتین نے فرانس کی مصنوعات کے با...   
20   موجودہ صفحہ, صفحۂ اول  جنیوا: کم سے کم اجرت کی بلند ترین سطح متعارف ک...   
21   موجودہ صفحہ, صفحۂ اول  یہ کیسے یقینی بنایا جائے کہ سرمایہ کاری کا دست...   
22   موجودہ صفحہ, صفحۂ اول  سات ماہ میں پہلی بار غیر ملکی مسلمانوں کا کعبہ...   
23   موجودہ صفحہ, صفحۂ اول  جب زمین پر بیٹھے سائنسدانوں نے خلائی جہاز کی م...   
24   موجودہ صفحہ, صفحۂ اول  افغانستان کی سڑکوں پر اشیا بیچنے والی لڑکیوں ک...   
25   موجودہ صفحہ, صفحۂ اول  امریکی صدارتی انتخاب میں کتنا خرچہ آتا ہے اور ...   
26                 پاکستان  جیل میں مشقت کرنے والا سرکش دانشور صحافی، سلیم...   
27                 پاکستان               بریگیڈیئر اعجاز شاہ اور آسیب کا سایہ   
28                 پاکستان  جب پاکستان کے پہلے صدر کو ’ملک کے مفاد میں‘ جل...   
29                 پاکستان  پاکستان سمیت مختلف ممالک کورونا وائرس کی ’دوسر...   
30                 پاکستان  یہ کیسے یقینی بنایا جائے کہ سرمایہ کاری کا دست...   
..                     ...                                                ...   
166                  سائنس  امریکہ سے نکالا جانے والا سائنسدان جس نے چین ک...   
167                  سائنس  چین کے 'کاربن سِنک' علاقے ماحول کے لیے کیسے مد...   
168                  سائنس  آپ کی جنسی زندگی کو بہتر بنانے کا وعدہ کرنے وا...   
169                  سائنس  پاکستان سمیت مختلف ممالک کورونا وائرس کی ’دوسر...   
170                  سائنس  مودی سے کنارہ کشی کرنے والا طوطا ’شرمیلا‘ تھا ...   
171                  سائنس  فرانس سے پاکستانی شہریوں کی مبینہ ملک بدری کی ...   
172                  سائنس        ویکسینز میں پائی جانے والی عجیب و غریب اشیا   
173                  سائنس  ’انھوں نے پوچھا کہ کیا ہم نے نیچے کپڑے پہن رکھ...   
174                  سائنس          ایسٹ انڈیا کمپنی جس نے ایک خطے پر راج کیا   
175                  سائنس  برطانوی وزیر اعظم بورس جانسن کا انگلینڈ میں دو...   
176                  سائنس                       غازی علم دین: ملزم اور مقدمہ   
177                  سائنس  ’اسقاطِ حمل نہ کروا کر میں نے اپنی زندگی کا بہ...   
178                  سائنس  امریکی خفیہ جغرافیائی معلومات انڈیا کے دفاعی ن...   
179                  سائنس                    کیا امریکہ واقعی آزاد ریاست ہے؟   
180                  سائنس  فرانسیسی مصنوعات کے بائیکاٹ کی لپیٹ میں ’پاکست...   
181                  سائنس  عالمی بحران کے باوجود بڑی ٹیکنالوجی کمپنیوں کا...   
182                  سائنس  راولاکوٹ: شہریوں کے تشدد کا نشانہ بننے والے سک...   
183                  سائنس  کورونا: پاکستان میں تین 

In [62]:
df.to_csv('Dataset.csv', encoding='utf-32', columns=['Label', 'Heading', 'Summary'], sep=';')